In [31]:
import tensorflow as tf
import numpy as np
import fasttext as ft
from helpers import *

# Load GloVe embedding from Stanford

In [4]:
f = open("glove_from_stanford/glove.twitter.27B.200d.txt", "r")
words = []

i = 0
#embeddings = np.zeros((1,200))
embeddings = []
for l in f.readlines():
    li = l.split()
    w = li[0]
    vec_string = li[1:]
    vec = []
    for e in vec_string:
        vec.append(float(e))
    vec = np.array(vec)
    if i%10000 == 0:
        print("done: ", i )
    if vec.shape[0] == 200:
        words.append(w)
        #embeddings = np.vstack((embeddings, vec))
        embeddings.append(vec)

    else:
        print(w, " was not the right shape. The shape was: ", vec.shape)
    i += 1
    
#embeddings = embeddings[1:,]

done:  0
done:  10000
done:  20000
done:  30000
-0.29736  was not the right shape. The shape was:  (199,)
done:  40000
done:  50000
done:  60000
done:  70000
done:  80000
done:  90000
done:  100000
done:  110000
done:  120000
done:  130000
done:  140000
done:  150000
done:  160000
done:  170000
done:  180000
done:  190000
done:  200000
done:  210000
done:  220000
done:  230000
done:  240000
done:  250000
done:  260000
done:  270000
done:  280000
done:  290000
done:  300000
done:  310000
done:  320000
done:  330000
done:  340000
done:  350000
done:  360000
done:  370000
done:  380000
done:  390000
done:  400000
done:  410000
done:  420000
done:  430000
done:  440000
done:  450000
done:  460000
done:  470000
done:  480000
done:  490000
done:  500000
done:  510000
done:  520000
done:  530000
done:  540000
done:  550000
done:  560000
done:  570000
done:  580000
done:  590000
done:  600000
done:  610000
done:  620000
done:  630000
done:  640000
done:  650000
done:  660000
done:  670000
done

In [5]:
embedding_stacked = np.stack(embeddings, axis=0)

In [16]:
words = np.array(words)

In [17]:
np.save("embedding_stanford", embedding_stacked)
np.save("stanford_words", words)

# Compute vectors for tweets

In [32]:
embedding = np.load("embedding_stanford.npy")
word_list = np.load("stanford_words.npy")

In [33]:
n_features = embedding.shape[1]

In [ ]:
tweets_pos_txt = []
f = open("Datasets/twitter-datasets/train_pos.txt")
for l in f.readlines():
    tweets_pos_txt.append(l.strip())
tweets_pos_txt = np.array(tweets_pos_txt)
print("begin means")
tweets_vecs_pos = tweet_means(tweets_pos_txt, embedding, word_list, n_features, False, {}, False, False, [])

begin means


In [ ]:
tweets_neg_txt = []
f = open("Datasets/twitter-datasets/train_neg.txt")
for l in f.readlines():
    tweets_neg_txt.append(l.strip())
tweets_neg_txt = np.array(tweets_neg_txt)
tweets_vecs_neg = tweet_means(tweets_neg_txt, embedding, word_list, n_features, False, {}, False, False, [])

In [15]:
X = np.load("tweets_X_sam.npy")
y = np.load("tweets_y_sam.npy")

j = 0.8

train_X = X[0:int(j*len(y))]
train_y = y[0:int(j*len(y))]
test_X = X[int(j*len(y)):]
test_y = y[int(j*len(y)):]

train_y[train_y == -1.] = 0.
test_y[test_y == -1.] = 0.

In [16]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_y))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(250, activation='relu'),
    tf.keras.layers.Dense(20, activation='softmax')
    #tf.keras.layers.Dense(50, activation='relu'),
    #tf.keras.layers.Dense(10, activation='softmax'),
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [33]:
model.fit(train_dataset, epochs=15)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/15
2500/2500 [==============================] - 10s 4ms/step - loss: 0.6495 - sparse_categorical_accuracy: 0.6164
Epoch 2/15
2500/2500 [==============================] - 6s 2ms/step - loss: 0.5826 - sparse_categorical_accuracy: 0.6760
Epoch 3/15
2500/2500 [==============================] - 5s 2ms/step - loss: 0.5495 - sparse_categorical_accuracy: 0.7039
Epoch 4/15
2500/2500 [==============================] - 5s 2ms/step - loss: 0.5265 - sparse_categorical_accuracy: 0.7218
Epoch 5/15
2500/2500 [=

In [34]:
model.evaluate(test_dataset)

625/625 [==============================] - 1s 1ms/step - loss: 0.8010 - sparse_categorical_accuracy: 0.6836


[0.8010182406425476, 0.6836]

In [78]:
fout = open("train_supervised.txt", "w")
fout2 = open("test_supervised.txt", "w")
f1 = open("Datasets/twitter-datasets/train_pos_clean.txt", "r")
f2 = open("Datasets/twitter-datasets/train_neg_clean.txt", "r")

i = 0
for l in f1.readlines():
    fout.write("__label__positive " + l)
    if i > 80000:
        fout2.write("__label__positive " + l)
    i += 1
i = 0
for l in f2.readlines():
    fout.write("__label__negative " + l)
    if i > 80000:
        fout2.write("__label__positive " + l)
    i+=1
f1.close()
f2.close()
fout.close()
fout2.close()

In [79]:
model = ft.train_supervised('train_supervised.txt')

In [80]:
model.labels

['__label__negative', '__label__positive']

In [81]:
model.predict("fuck")

(('__label__positive',), array([0.6640566]))

In [82]:
model.test("test_supervised.txt", k=1)

(21319, 0.5366574417186547, 0.5366574417186547)

array([ 0.02571399,  0.15396671,  0.04118297, -0.21249573,  0.10300473,
        0.0679355 , -0.09303714,  0.02835391,  0.16474135,  0.16493794,
       -0.02768764, -0.07995272, -0.09622344,  0.09716307, -0.09126273,
       -0.06234534,  0.08886703,  0.18623899, -0.15840052, -0.04535728,
       -0.09760258,  0.15655494,  0.04274376,  0.0777818 , -0.04115377,
       -0.20150737, -0.09738524, -0.11599219, -0.17654024,  0.15988488,
       -0.17184366,  0.10775842, -0.09421516, -0.13489622, -0.27430215,
       -0.08880118, -0.08983865, -0.0888577 ,  0.06397012, -0.01490111,
       -0.07386695,  0.31255385,  0.06257832,  0.11577053,  0.13096482,
       -0.05254146,  0.12596858, -0.10450681, -0.10654113,  0.24910197,
       -0.12927958,  0.18731278, -0.11526599,  0.14024238, -0.14922841,
        0.03247508, -0.18863636, -0.09864707,  0.10644189, -0.02615612,
        0.00728449,  0.10776213,  0.15437657,  0.13160653, -0.01349203,
       -0.09614967, -0.20270133, -0.02062804,  0.00064062, -0.10